In [1]:
import numpy as np
import pandas as pd
from sklearn import svm, neighbors, linear_model, neural_network
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.mixture import GaussianMixture
from sklearn.covariance import GraphicalLasso
from sklearn.svm import SVC, NuSVC
from sklearn.pipeline import Pipeline
from tqdm import tqdm_notebook
import warnings
import multiprocessing
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import manifold
from scipy.optimize import minimize  
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
cols = [c for c in train.columns if c not in ['id', 'target', 'wheezy-copper-turtle-magic']]
print(train.shape, test.shape)

(262144, 258) (131073, 257)


In [3]:
def get_mean_cov(x, y, component_num):
    np.random.seed(42)
    ones = (y == 1).astype(bool)
    x2 = x[ones]
    gmm_1 = GaussianMixture(n_components=component_num)
    gmm_1.fit(x2)
    m1 = gmm_1.means_
    p1 = gmm_1.precisions_

    zeros = (y == 0).astype(bool)
    x2b = x[zeros]
    gmm_0 = GaussianMixture(n_components=component_num)
    gmm_0.fit(x2b)
    m2 = gmm_0.means_
    p2 = gmm_0.precisions_

    ms = np.concatenate((m1, m2), axis=0)
    ps = np.concatenate((p1, p2), axis=0)

    return ms, ps

In [4]:
np.random.seed(42)
oof_gmm = np.zeros(len(train))
oof_gmm_2 = np.zeros(len(train))
oof_gmm_3 = np.zeros(len(train))
preds_gmm = np.zeros(len(test))
records_gmm = []

# 512 models
for i in tqdm_notebook(range(512)):

    train2 = train[train['wheezy-copper-turtle-magic'] == i]
    test2 = test[test['wheezy-copper-turtle-magic'] == i]
    idx1 = train2.index
    idx2 = test2.index
    train2.reset_index(drop=True, inplace=True)

    data = pd.concat([pd.DataFrame(train2[cols]), pd.DataFrame(test2[cols])])
    pipe = Pipeline([('vt', VarianceThreshold(threshold=2)),
                     ('scaler', StandardScaler())])
    data2 = pipe.fit_transform(data[cols])
    train3 = data2[:train2.shape[0]]
    test3 = data2[train2.shape[0]:]
    
    skf = StratifiedKFold(n_splits=15, random_state=42,shuffle=True)

    for train_index, test_index in skf.split(train2, train2['target']):

        #######################################################################################
        cluster_per_class = 2
        ms, ps = get_mean_cov(train3[train_index,:],train2.loc[train_index]['target'].values,cluster_per_class)
        
        gmm_2 = GaussianMixture(n_components=cluster_per_class * 2, init_params='kmeans', covariance_type='full', max_iter=100, n_init=1,means_init=ms, precisions_init=ps)
        gmm_2.fit(np.concatenate([train3,test3],axis = 0))
        
        prob_2 = gmm_2.predict_proba(train3[test_index,:])
        prob_class_2 = np.zeros(prob_2.shape[0])

        for j in range(prob_2.shape[0]):
            if(np.argmax(prob_2,axis=1)[j] in np.arange(cluster_per_class)):
                prob_class_2[j] = np.max(prob_2,axis=1)[j]
            else:
                prob_class_2[j] = 1 - np.max(prob_2,axis=1)[j]
                
        #######################################################################################
        cluster_per_class = 3
        ms, ps = get_mean_cov(train3[train_index,:],train2.loc[train_index]['target'].values,cluster_per_class)
        
        gmm_3 = GaussianMixture(n_components=cluster_per_class * 2, init_params='kmeans', covariance_type='full', max_iter=100, n_init=1,means_init=ms, precisions_init=ps)
        gmm_3.fit(np.concatenate([train3,test3],axis = 0))
        
        prob_3 = gmm_3.predict_proba(train3[test_index,:])
        prob_class_3 = np.zeros(prob_3.shape[0])

        for j in range(prob_3.shape[0]):
            if(np.argmax(prob_3,axis=1)[j] in np.arange(cluster_per_class)):
                prob_class_3[j] = np.max(prob_3,axis=1)[j]
            else:
                prob_class_3[j] = 1 - np.max(prob_3,axis=1)[j]
        
        #######################################################################################
        oof_gmm_2[idx1[test_index]] = prob_class_2
        oof_gmm_3[idx1[test_index]] = prob_class_3
        
    gmm_2_score = roc_auc_score(train['target'][idx1].values, oof_gmm_2[idx1])
    gmm_3_score = roc_auc_score(train['target'][idx1].values, oof_gmm_3[idx1])
    gmm_score = [gmm_2_score,gmm_3_score]
    
    if(np.argmax(gmm_score) == 0):
        cluster_per_class = 2
    else:
        cluster_per_class = 3

    for train_index, test_index in skf.split(train2, train2['target']):

        ms, ps = get_mean_cov(train3[train_index,:],train2.loc[train_index]['target'].values, cluster_per_class)
        gmm = GaussianMixture(n_components=cluster_per_class * 2, init_params='kmeans', covariance_type='full', max_iter=100, n_init=1,means_init=ms, precisions_init=ps)
        gmm.fit(np.concatenate([train3,test3],axis = 0))
        
        prob = gmm.predict_proba(train3[test_index,:])
        prob_class = np.zeros(prob.shape[0])

        for j in range(prob.shape[0]):
            if(np.argmax(prob,axis=1)[j] in np.arange(cluster_per_class)):
                prob_class[j] = np.max(prob,axis=1)[j]
            else:
                prob_class[j] = 1 - np.max(prob,axis=1)[j]

        oof_gmm[idx1[test_index]] = prob_class
        
        prob_test = gmm.predict_proba(test3)
        prob_test_class = np.zeros(prob_test.shape[0])
        for j in range(prob_test.shape[0]):
            if(np.argmax(prob_test,axis=1)[j] in np.arange(cluster_per_class)):
                prob_test_class[j] = np.max(prob_test,axis=1)[j]
            else:
                prob_test_class[j] = 1 - np.max(prob_test,axis=1)[j]

        preds_gmm[idx2] += prob_test_class / skf.n_splits
        
    print(i, cluster_per_class, roc_auc_score(train['target'][idx1].values, oof_gmm_2[idx1]),roc_auc_score(train['target'][idx1].values, oof_gmm_3[idx1]),roc_auc_score(train['target'][idx1].values, oof_gmm[idx1]))
    records_gmm.append(roc_auc_score(train['target'][idx1].values, oof_gmm[idx1]))
        
auc = roc_auc_score(train['target'], oof_gmm)
print(f'AUC: {auc:.5}')

0 2 0.9682558612658371 0.9614159639695256 0.9682558612658371
1 2 0.973631128356738 0.9679647080561714 0.973631128356738
2 2 0.9741083676268861 0.9617381932196748 0.9741083676268861
3 2 0.9854189456776106 0.9852389326612847 0.9854189456776106
4 2 0.9801217038539554 0.9771542333408458 0.9801217038539554
5 2 0.969546700462731 0.9692667039995284 0.969546700462731
6 2 0.9893877246625399 0.9839361622790663 0.9893877246625399
7 3 0.9593019622547181 0.9599268841394826 0.9599268841394826
8 2 0.9733494896085776 0.9715576118087685 0.9733494896085776
9 2 0.9742022497704317 0.9722724403122132 0.9742022497704317
10 2 0.9704639308297844 0.965310040005162 0.9704639308297844
11 2 0.9813275862068966 0.9756120689655173 0.9813275862068966
12 2 0.975355719977234 0.9748922676640378 0.975355719977234
13 3 0.9789446808510638 0.9832680851063829 0.9832680851063829
14 2 0.9910665714674284 0.9775091799265607 0.9910665714674284
15 3 0.9690331304935766 0.9763503868980543 0.9763503868980543
16 3 0.975294311475901 0.

In [5]:
sub = pd.read_csv('../input/sample_submission.csv')
sub['target'] = preds_gmm
sub.to_csv('submission.csv',index=False)

In [6]:
# for itr in range(1):
#     test['target'] = preds_gmm
#     test.loc[test['target'] > 0.95, 'target'] = 1
#     test.loc[test['target'] < 0.05, 'target'] = 0
#     usefull_test = test[(test['target'] == 1) | (test['target'] == 0)]
#     new_train = pd.concat([train, usefull_test]).reset_index(drop=True)
#     print(usefull_test.shape[0], "Test Records added for iteration : ", itr)
    
#     new_train.loc[oof_gmm > 0.98, 'target'] = 1
#     new_train.loc[oof_gmm < 0.02, 'target'] = 0
    
#     oof_gmm2 = np.zeros(len(train))
#     preds_gmm = np.zeros(len(test))
    
#     for i in tqdm_notebook(range(512)):

#         train2 = train[train['wheezy-copper-turtle-magic'] == i]
#         test2 = test[test['wheezy-copper-turtle-magic'] == i]
#         idx1 = train2.index
#         idx2 = test2.index
#         train2.reset_index(drop=True, inplace=True)

#         data = pd.concat([pd.DataFrame(train2[cols]), pd.DataFrame(test2[cols])])
#         pipe = Pipeline([('vt', VarianceThreshold(threshold=2)),
#                          ('scaler', StandardScaler())])
#         data2 = pipe.fit_transform(data[cols])
#         train3 = data2[:train2.shape[0]]
#         test3 = data2[train2.shape[0]:]

#         skf = StratifiedKFold(n_splits=11, random_state=2019,shuffle=True)

#         for train_index, test_index in skf.split(train2, train2['target']):

#             # MODEL AND PREDICT WITH QDA
#             ms, ps = get_mean_cov(train3[train_index,:],train2.loc[train_index]['target'].values)

#             gmm = GaussianMixture(n_components=component_num*2, init_params='kmeans', covariance_type='full', max_iter=100, n_init=1,means_init=ms, precisions_init=ps)
#             gmm.fit(np.concatenate([train3,test3],axis = 0))

#             prob = gmm.predict_proba(train3[test_index,:])
#             prob_class = np.zeros(prob.shape[0])

#             for j in range(prob.shape[0]):
#                 if(np.argmax(prob,axis=1)[j] in {0, 1}):
#                     prob_class[j] = np.max(prob,axis=1)[j]
#                 else:
#                     prob_class[j] = 1 - np.max(prob,axis=1)[j]

#             oof_gmm2[idx1[test_index]] = prob_class

# #             prob_test = gmm.predict_proba(test3)
# #             prob_test_class = np.zeros(prob_test.shape[0])
# #             for j in range(prob_test.shape[0]):
# #                 if(np.argmax(prob_test,axis=1)[j] in {0, 1, 2}):
# #                     prob_test_class[j] = np.max(prob_test,axis=1)[j]
# #                 else:
# #                     prob_test_class[j] = 1 - np.max(prob_test,axis=1)[j]

# #             preds_gmm[idx2] += prob_test_class / skf.n_splits
    
#     auc = roc_auc_score(train['target'], oof_gmm2)
#     print(f'AUC: {auc:.5}')

In [7]:
# i=0
# train2 = train[train['wheezy-copper-turtle-magic'] == i]
# test2 = test[test['wheezy-copper-turtle-magic'] == i]
# idx1 = train2.index
# idx2 = test2.index
# train2.reset_index(drop=True, inplace=True)

# data = pd.concat([pd.DataFrame(train2[cols]), pd.DataFrame(test2[cols])])
# pipe = Pipeline([('vt', VarianceThreshold(threshold=2)),
#                  ('scaler', StandardScaler())])
# data2 = pipe.fit_transform(data[cols])
# train3 = data2[:train2.shape[0]]
# test3 = data2[train2.shape[0]:]

# print(train2.shape)
# print(train3.shape)
# print(train2['target'].shape)

# np.random.seed(42)
# for train_index, test_index in skf.split(train2, train2['target']):

#     # MODEL AND PREDICT WITH QDA
#     ms, ps = get_mean_cov(train3[train_index,:],train2.loc[train_index]['target'].values)

#     gmm = GaussianMixture(n_components=6, init_params='random', covariance_type='full', tol=0.001,reg_covar=0.001, max_iter=200, n_init=1,means_init=ms, precisions_init=ps)
#     gmm.fit(np.concatenate([train3,test3],axis = 0))
#     prob = gmm.predict_proba(train3[test_index,:])
#     prob_class = np.zeros(prob.shape[0])
    
#     for j in range(prob.shape[0]):
#         if(np.argmax(prob,axis=1)[j] in {0,1,2}):
#             prob_class[j] = np.max(prob,axis=1)[j]
#         else:
#             prob_class[j] = 1 - np.max(prob,axis=1)[j]
    
#     oof_gmm[idx1[test_index]] = prob_class
# #     preds_gmm[idx2] += gmm.predict_proba(test3)[:,0] / skf.n_splits
# print(roc_auc_score(train['target'][idx1].values, oof_gmm[idx1]))